In [47]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.callbacks import ModelCheckpoint
#from keras.optimizers import Adam
from keras import optimizers

In [48]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [49]:
# create model
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(6,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)

In [50]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies
#train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
#                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
#                                   horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('./merged_full_split_final/train/', # this is where you specify the path to the main data folder
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_generator = test_datagen.flow_from_directory(
    directory='./merged_full_split_final/val/',
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 3301 images belonging to 6 classes.
Found 826 images belonging to 6 classes.


In [51]:
# load weights
model.load_weights("./mobilenet_best_models_100_epochs/weights-improvement-34-0.79.hdf5")
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size
step_size_test=test_generator.n//test_generator.batch_size
history = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=0,
                   validation_data=test_generator,
                   validation_steps=step_size_test)
model.save("79_accuracy_not_augmented.h5")


In [43]:
model.output

<tf.Tensor 'dense_16/Softmax:0' shape=(?, 6) dtype=float32>

In [44]:
history

In [1]:
model.summary()

NameError: name 'model' is not defined